In [1]:
import torch
from torch import nn

In [2]:
import sys
sys.path.append('..')
from common import d2l

#### 1. 多输入通道

In [3]:
def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [4]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

In [7]:
X, K, corr2d_multi_in(X, K)

(tensor([[[0., 1., 2.],
          [3., 4., 5.],
          [6., 7., 8.]],
 
         [[1., 2., 3.],
          [4., 5., 6.],
          [7., 8., 9.]]]),
 tensor([[[0., 1.],
          [2., 3.]],
 
         [[1., 2.],
          [3., 4.]]]),
 tensor([[ 56.,  72.],
         [104., 120.]]))

#### 2. 多输出通道

In [13]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [9]:
K = torch.stack((K, K + 1, K + 2), 0)

In [11]:
K, K.shape

(tensor([[[[0., 1.],
           [2., 3.]],
 
          [[1., 2.],
           [3., 4.]]],
 
 
         [[[1., 2.],
           [3., 4.]],
 
          [[2., 3.],
           [4., 5.]]],
 
 
         [[[2., 3.],
           [4., 5.]],
 
          [[3., 4.],
           [5., 6.]]]]),
 torch.Size([3, 2, 2, 2]))

In [15]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

#### 3. 1 × 1 卷积核

In [29]:
def corr2d_multi_in_out_1x1(X, K):
    """
    使用全连接实现1 × 1卷积
    """
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [18]:
X = torch.normal(0, 1, (3, 3, 3))
X

tensor([[[ 0.7618,  0.5524, -1.4748],
         [ 0.1395, -0.0662, -2.1202],
         [ 0.9628, -1.6372, -0.5834]],

        [[ 0.9430,  0.6866, -0.6623],
         [-0.1969,  0.2370,  1.2816],
         [ 1.0561, -2.6448,  0.7032]],

        [[ 1.3377,  1.0728, -0.1726],
         [-0.5431,  0.0862, -0.1626],
         [ 0.6299,  1.0852, -0.5187]]])

In [21]:
K = torch.normal(0, 1, (2, 3, 1, 1))
K

tensor([[[[-0.2518]],

         [[-1.4198]],

         [[-1.0211]]],


        [[[ 1.5858]],

         [[-0.9470]],

         [[-0.9477]]]])

In [23]:
Y1 = corr2d_multi_in_out(X, K)
Y1

tensor([[[-2.8966, -2.2094,  1.4879],
         [ 0.7990, -0.4079, -1.1199],
         [-2.3851,  3.0592, -0.3219]],

        [[-0.9526, -0.7908, -1.5479],
         [ 0.9223, -0.4111, -4.4218],
         [-0.0703, -1.1202, -1.0995]]])

In [30]:
Y2 = corr2d_multi_in_out_1x1(X, K)
Y2

tensor([[[-2.8966, -2.2094,  1.4879],
         [ 0.7990, -0.4079, -1.1199],
         [-2.3851,  3.0592, -0.3219]],

        [[-0.9526, -0.7908, -1.5479],
         [ 0.9223, -0.4111, -4.4218],
         [-0.0703, -1.1202, -1.0995]]])

In [31]:
torch.abs(Y1 - Y2)

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])